In [1]:
%pip install fastparquet

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
BANNER = """
MMWXkolccc:::::::::::::::::::::cclokXWMM
MXd,.                              .,dXM
Xc.                            ..... .cX
k.                            .',;,.. .k
d.    ....................... .',;,.. .d
d.    'dOOOOOOOOOOOOOOOOOOOkdc'.....  .d
d.    ,0MMMMMMMMMMMMMMMMMMMMMWO;.     .d
d.    ,0MMMXdcccccccccccclOWMMWd.     .d
d.    ,0MMMO'             cXMMWx.     .d
d.    ,0MMMXocccccccccccclkWMMWd.     .d
d.    ,0MMMMMWWWWWWWWMMWMMMMMWK:.     .d
d.    ,0MMMWK000000000XWMMMNOo,.      .d
d.    ,0MMM0;.........,xNMMXo.        .d
d.    ,0MMM0,          .lKMMWO;.      .d
d.    'kXXXk'           .;kXXXO:.     .d
x.    ..'''..             .''''..     .x
0,                                    ,0
Wk,.                                .,kW
MWKd:'............................':dKWM
MMMMNKOxxddddddddddddddddddddddxxOKNMMMM
"""

In [3]:
import faiss
from faiss import write_index, read_index
from sentence_transformers import SentenceTransformer
import math
import pandas as pd
import requests
from urllib.parse import urlencode
import os
import numpy as np
from catboost import CatBoostRanker, Pool
from functools import partial
import json
import pyarrow.parquet as pq
import time
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

# Загрузка данных

In [4]:
start_time = time.time()

In [5]:
def download_file(pk, local_name):
    base_url = \
        'https://cloud-api.yandex.net/v1/disk/public/resources/download?'

    final_url = base_url + urlencode(dict(public_key=pk))
    response = requests.get(final_url)
    download_url = response.json()['href']

    download_response = requests.get(download_url)
    with open(local_name, 'wb') as f:
        f.write(download_response.content)
        print(f'File {local_name} downloaded')


def download_from_yandex_disk():
    print(BANNER)

    files = {
        'features.parquet': 'https://disk.yandex.ru/d/W_qJitz4dZGzAg',
        'videos.parquet': 'https://disk.yandex.ru/d/JXz-oDfKFgm2Dw',
        'automarkup.parquet': 'https://disk.yandex.ru/d/vP0FzQHdtxsz4Q',
        'manualmarkup.csv': 'https://disk.yandex.ru/d/hDztN1rgW0JNjw'
    }

    filenames, filenames_to_delete = files.keys(), []
    for filename in filenames:
        if os.path.exists(filename):
            filenames_to_delete += [filename]
    for filename in filenames_to_delete:
        del files[filename]

    print(f'Will be download {len(files)} files')
    for filename, link in files.items():
        print(f'{filename} downloading in progress')
        download_file(link, filename)
        print(f'{filename} downloaded')

In [6]:
download_from_yandex_disk()


MMWXkolccc:::::::::::::::::::::cclokXWMM
MXd,.                              .,dXM
Xc.                            ..... .cX
k.                            .',;,.. .k
d.    ....................... .',;,.. .d
d.    'dOOOOOOOOOOOOOOOOOOOkdc'.....  .d
d.    ,0MMMMMMMMMMMMMMMMMMMMMWO;.     .d
d.    ,0MMMXdcccccccccccclOWMMWd.     .d
d.    ,0MMMO'             cXMMWx.     .d
d.    ,0MMMXocccccccccccclkWMMWd.     .d
d.    ,0MMMMMWWWWWWWWMMWMMMMMWK:.     .d
d.    ,0MMMWK000000000XWMMMNOo,.      .d
d.    ,0MMM0;.........,xNMMXo.        .d
d.    ,0MMM0,          .lKMMWO;.      .d
d.    'kXXXk'           .;kXXXO:.     .d
x.    ..'''..             .''''..     .x
0,                                    ,0
Wk,.                                .,kW
MWKd:'............................':dKWM
MMMMNKOxxddddddddddddddddddddddxxOKNMMMM

Will be download 0 files


In [7]:
seed = 42
np.random.seed(seed)

# Формирование базы векторов

In [8]:
candidates = pd.read_parquet(
    'videos.parquet',
    engine='fastparquet',
    columns=['video_id', 'video_title']
)

In [9]:
candidates = candidates.sample(n=7_000_000, replace=False, random_state=seed)

In [10]:
corpus = candidates['video_title'].apply(lambda x: x.lower()).values
video_ids = candidates['video_id'].values
del candidates

In [11]:
st_model = SentenceTransformer(
    'cointegrated/rubert-tiny2',
    device='cuda'
)

.gitattributes:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

.gitignore:   0%|          | 0.00/6.00 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

...

tinybert-ru-labse-adapter-v2.pt:   0%|          | 0.00/963k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.74M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/401 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [12]:
# когда прогоните один раз у вас на диске уже будет сохранен faiss индекс
# можно поставить значение True, чтобы сэкономить время на формирование индекса
use_formed_index = True

In [13]:
d = 312
if not use_formed_index:
    cpu_index = faiss.IndexFlatL2(d)
    cpu_index.is_trained, cpu_index.ntotal

In [24]:
# если уже есть файл ind2videoid для вашего faiss индекса - True
use_formed_id_mapping = False

if not use_formed_id_mapping:
    ind2videoid = {ind: video_id for ind, video_id in enumerate(video_ids)}
    with open('ind2videoid.json', 'w+') as f:
        json.dump(ind2videoid, f, indent=4)
else:
    with open('ind2videoid.json', 'r') as f:
        ind2videoid = json.load(f)

In [15]:
batch_size = 100000
num_batches = math.ceil(len(corpus) / batch_size)

In [16]:
if not use_formed_index:
    try:
        for i in range(num_batches):
            # формируем батч
            start, end = i * batch_size, (i + 1) * batch_size
            corpus_batch = corpus[start:end]

            # считаем вектора для всех предложений в батче
            embeddings = st_model.encode(
                corpus_batch,
                batch_size=1000,
                show_progress_bar=True
            )

            # добавляем новые батч векторов в индекс и сохраняем его
            cpu_index.add(embeddings)
            write_index(cpu_index, 'candidates.index')

            print(f'batch: {i + 1} / {num_batches}, vectors: {cpu_index.ntotal}')

            # чистим ОЗУ
            del embeddings
    except KeyboardInterrupt:
        print('Остановлено пользователем')
        try:
            del embeddings
        except:
            pass

In [17]:
# чистим ОЗУ
if not use_formed_index:
    del cpu_index

# Подбор кандидатов по базе векторов

In [62]:
automarkup = pd.read_parquet(
    'automarkup.parquet',
    engine='fastparquet'
)

In [19]:
automarkup = automarkup[~automarkup['query'].isna()]
automarkup['query'] = automarkup['query'].apply(lambda x: x.lower())

In [100]:
n = 200
top_n = automarkup['query'].value_counts()[:int(2*n)].index.to_list()
other = np.array(automarkup['query'].value_counts()[int(2*n):].index.to_list())
random_n = np.random.choice(other, size=n, replace=False).tolist()
queries = top_n + random_n
print(type(queries))
print(len(queries))
query2ind = {q: i for i, q in enumerate(queries)}

<class 'list'>
600


In [56]:
# когда прогоните один раз у вас на диске кандиды уже будут сохранены
# можете поставить значение True, чтобы сэкономить время
use_formed_candidates = False

In [57]:
qembeddings = st_model.encode(
    queries,
    batch_size=100,
    show_progress_bar=True
)

search_cpu_index = read_index('candidates.index')
search_cpu_index.is_trained, search_cpu_index.ntotal

NameError: name 'st_model' is not defined

In [58]:
ind2videoid = pd.DataFrame.from_dict(ind2videoid, orient='index')
print(type(ind2videoid))

TypeError: 'numpy.ndarray' object is not callable

In [59]:
print(ind2videoid.columns)
for col in ind2videoid.columns:
    print(col)
    
a = ind2videoid[ind2videoid[0] == "video_24972264"]
print(ind2videoid.iloc[0,0])


RangeIndex(start=0, stop=1, step=1)
0
video_24972264


In [60]:
generated_cand_name = 'generated_candidates.parquet'

if not use_formed_candidates:

    topk = 10
    distance, faiss_ind = search_cpu_index.search(qembeddings, topk)

    generated_cand = {
        'query': [],
        'video_id': []
    }
    

    """def loop_1(queries):"""

    for i, q in enumerate(queries):
        print(f"Epoch {i}")
        vids = faiss_ind[i]
        for v in vids:
            vid = ind2videoid.iloc[v,0]
            generated_cand['video_id'].append(vid)
        
            """[list(ind2videoid.keys())[list(ind2videoid.values()).index(str(v))] for v in vids]"""
        for j in range(len(vids)):
            generated_cand['query'].append(q)
            
    """with ThreadPoolExecutor(workers) as executor:
        _ = list(tqdm(executor.map(loop_1, queries), total=len(queries)))"""
    
    """print(search_cpu_index[0])
    top_k = 30
    for query in queries:
        query_embedding = embedder.encode(query)

        # We use cosine-similarity and torch.topk to find the highest 5 scores
        cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
        top_results = torch.topk(cos_scores, k=top_k)"""
    
    print("Done")

    print(generated_cand["video_id"])
    print(generated_cand["query"])
    
    generated_cand = pd.DataFrame(generated_cand)

    generated_cand.to_parquet(
        generated_cand_name,
        engine='fastparquet'
    )
else:
    generated_cand = pd.read_parquet(
        generated_cand_name,
        engine='fastparquet'
    )

# Формирование таргета по авторазметке

In [80]:
automarkup['target'] = [1] * automarkup.shape[0]
candidates_with_target = generated_cand.merge(
    automarkup[['query', 'video_id','target']],
    how='left',
    left_on=['query', 'video_id'],
    right_on=['query', 'video_id']
)
print(candidates_with_target.columns)
print(candidates_with_target.target.unique())
check = candidates_with_target.loc[candidates_with_target["target"] == 1.0]
print(check)
candidates_with_target['target'] = candidates_with_target['target'].fillna(0)

check = candidates_with_target.loc[candidates_with_target["target"] == 1.0]

                                       query        video_id  target
32                           звезды в африке  video_12736047     1.0
33                           звезды в африке  video_12736047     1.0
34                           звезды в африке  video_12736047     1.0
43                        битва экстрасенсов   video_3811634     1.0
82                     новые звезды в африке   video_5975922     1.0
...                                      ...             ...     ...
9330         беременна в 16 однажды в россии  video_29223287     1.0
9331         беременна в 16 однажды в россии  video_29223287     1.0
9332         беременна в 16 однажды в россии  video_29223287     1.0
9333         беременна в 16 однажды в россии  video_29223287     1.0
9484  мать героиня наследие мужское женское   video_16886162     1.0

[3588 rows x 3 columns]


In [55]:
# очищаем ОЗУ
"""del generated_cand
del search_cpu_index
del automarkup
del st_model
del qembeddings"""

In [84]:
# здесь мы используем признаки актуальные на состояние 2-ого мая 2023 года
# вы можете использовать признаки наиболее актуальные для вашего кандидата

features_parquet = pq.ParquetFile('features.parquet')
features, filter_date = None, '2023-10-02'

for batch in features_parquet.iter_batches():
    tmp = batch.to_pandas()
    if features is None:
        features = tmp[tmp['report_date'] >= filter_date]
    else:
        features = pd.concat([
            features, 
            tmp[tmp['report_date'] == filter_date]
        ], axis=0)

# для baseline выбросим категориальные признаки и datetime признаки
# в своем решении вы сами можете решить использовать их или нет
features = features.drop(
    [
        'v_channel_reg_datetime',
        'v_channel_type',
        'v_pub_datetime'
    ], 
    axis=1
)

# Формирование датасета с признаками

In [85]:
full_df = candidates_with_target.merge(
    features,
    how='inner',
    left_on='video_id',
    right_on='video_id'
)
"""del features"""
print(full_df)
full_df = full_df.drop('report_date', axis=1)
full_df = full_df.drop_duplicates()
print(full_df)

                                                  query  ... v_cr_click_comment_1_days
0                                      битва сильнейших  ...                       0.0
1                          экстрасенсы битва сильнейших  ...                       0.0
2                                     битва сильнейших   ...                       0.0
3                         экстрасенсы. битва сильнейших  ...                       0.0
4                         экстрасенсы битва сильнейших   ...                       0.0
...                                                 ...  ...                       ...
6545  четыре свадьбы дворянская свадьба vs свадьба в...  ...                       0.0
6546  четыре свадьбы дворянская свадьба vs свадьба в...  ...                       0.0
6547                                         шоурум пик  ...                       0.0
6548                                         шоурум пик  ...                       0.0
6549                                 тилька

In [86]:
groups_to_drop = []
full_df['group_id'] = full_df.groupby(['query']).ngroup()
for group in full_df['group_id'].unique():
    part_df = full_df[full_df['group_id'] == group]
    target_sum = part_df['target'].values.sum()
    if target_sum <= 0:
        groups_to_drop += [group]
print(full_df)
"""full_df = full_df[~full_df['group_id'].isin(groups_to_drop)]"""
print(full_df)

                                                  query  ... group_id
0                                      битва сильнейших  ...       87
1                          экстрасенсы битва сильнейших  ...      581
2                                     битва сильнейших   ...       88
3                         экстрасенсы. битва сильнейших  ...      588
4                         экстрасенсы битва сильнейших   ...      582
...                                                 ...  ...      ...
6545  четыре свадьбы дворянская свадьба vs свадьба в...  ...      559
6546  четыре свадьбы дворянская свадьба vs свадьба в...  ...      559
6547                                         шоурум пик  ...      574
6548                                         шоурум пик  ...      574
6549                                 тилька бани гренни  ...      515

[3308 rows x 28 columns]
                                                  query  ... group_id
0                                      битва сильнейших  ...    

In [87]:
print(full_df)
groups = pd.Series(full_df['group_id'].unique())
permutation = groups.sample(frac=1, random_state=seed)
train_groups, val_groups, test_groups = np.split(
    permutation,
    [int(0.75 * len(permutation)), int(0.90 * len(permutation))]
)
print(train_groups)
print(val_groups)
print(test_groups)
print(full_df)

                                                  query  ... group_id
0                                      битва сильнейших  ...       87
1                          экстрасенсы битва сильнейших  ...      581
2                                     битва сильнейших   ...       88
3                         экстрасенсы. битва сильнейших  ...      588
4                         экстрасенсы битва сильнейших   ...      582
...                                                 ...  ...      ...
6545  четыре свадьбы дворянская свадьба vs свадьба в...  ...      559
6546  четыре свадьбы дворянская свадьба vs свадьба в...  ...      559
6547                                         шоурум пик  ...      574
6548                                         шоурум пик  ...      574
6549                                 тилька бани гренни  ...      515

[3308 rows x 28 columns]
188    223
238    214
462    484
465    452
182    370
      ... 
85     376
242    288
186    459
326    169
159    268
Length: 446, 

In [88]:
train_df = full_df[full_df['group_id'].isin(train_groups)]
print(train_df)
val_df = full_df[full_df['group_id'].isin(val_groups)]
print(val_df)
test_df = full_df[full_df['group_id'].isin(test_groups)]
print(test_df)

                                                  query  ... group_id
0                                      битва сильнейших  ...       87
2                                     битва сильнейших   ...       88
3                         экстрасенсы. битва сильнейших  ...      588
5                             битва сильнейших 3 выпуск  ...       94
6                             битва сильнейших 4 выпуск  ...       96
...                                                 ...  ...      ...
6545  четыре свадьбы дворянская свадьба vs свадьба в...  ...      559
6546  четыре свадьбы дворянская свадьба vs свадьба в...  ...      559
6547                                         шоурум пик  ...      574
6548                                         шоурум пик  ...      574
6549                                 тилька бани гренни  ...      515

[2509 rows x 28 columns]
                                       query  ... group_id
4              экстрасенсы битва сильнейших   ...      582
20              

In [89]:
train_df = train_df.sort_values('group_id')
val_df = val_df.sort_values('group_id')
test_df = test_df.sort_values('group_id')

In [90]:
metainfo_columns = ['query', 'video_id', 'target', 'group_id', 'v_category']

X_train = train_df.drop(metainfo_columns, axis=1)
y_train, g_train = train_df['target'], train_df['group_id']

print(X_train.columns.to_list())
print(y_train)

X_val = val_df.drop(metainfo_columns, axis=1)
y_val, g_val = val_df['target'], val_df['group_id']

X_test = test_df.drop(metainfo_columns, axis=1)
y_test, g_test = test_df['target'], test_df['group_id']

['total_comments', 'v_year_views', 'v_month_views', 'v_week_views', 'v_day_views', 'v_likes', 'v_dislikes', 'v_duration', 'v_cr_click_like_7_days', 'v_cr_click_dislike_7_days', 'v_cr_click_vtop_7_days', 'v_cr_click_long_view_7_days', 'v_cr_click_comment_7_days', 'v_cr_click_like_30_days', 'v_cr_click_dislike_30_days', 'v_cr_click_vtop_30_days', 'v_cr_click_long_view_30_days', 'v_cr_click_comment_30_days', 'v_cr_click_like_1_days', 'v_cr_click_dislike_1_days', 'v_cr_click_vtop_1_days', 'v_cr_click_long_view_1_days', 'v_cr_click_comment_1_days']
4874    0.0
4873    0.0
4871    0.0
4872    0.0
4869    0.0
       ... 
1890    0.0
1891    0.0
132     0.0
1889    0.0
128     0.0
Name: target, Length: 2509, dtype: float64


In [91]:
train = Pool(
    data=X_train.values,
    label=y_train.values,
    group_id=g_train.values,
    feature_names=X_train.columns.to_list()
)

val = Pool(
    data=X_val.values,
    label=y_val.values,
    group_id=g_val.values,
    feature_names=X_val.columns.to_list()
)

test = Pool(
    data=X_test.values,
    label=y_test.values,
    group_id=g_test.values,
    feature_names=X_test.columns.to_list()
)

# Обучение модели

In [92]:
task_type = 'GPU'
metric_period = 250

parameters = {
    'task_type': task_type,
    'verbose': False,
    'random_seed': seed,
    'loss_function': 'QueryRMSE',
    'learning_rate': 0.001,
    'l2_leaf_reg': 30,
    'iterations': 4000,
    'max_depth': 3,
}

In [93]:
model = CatBoostRanker(**parameters)
model = model.fit(
    train,
    eval_set=val,
    plot=True,
    use_best_model=True,
    metric_period=metric_period
)
model.save_model('ranker.ckpt')

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

# Измерение метрик

In [94]:
def _metrics_at(at, model, pool, metric='NDCG'):
    metric = metric + f':top={at}'
    eval_metrics = model.eval_metrics(pool, metrics=[metric])
    best_metrics = {}
    for key in eval_metrics.keys():
        best_metrics[key] = eval_metrics[key][model.best_iteration_]
    return best_metrics


metrics_train_at = partial(
    _metrics_at,
    model=model, 
    pool=train
)

metrics_val_at = partial(
    _metrics_at, 
    model=model, 
    pool=val
)

metrics_test_at = partial(
    _metrics_at,
    model=model, 
    pool=test
)

In [95]:
metrics_train_at(1), metrics_train_at(5)

({'NDCG:top=1;type=Base': 0.9932735426008968},
 {'NDCG:top=5;type=Base': 0.995882954563868})

In [96]:
metrics_val_at(1), metrics_val_at(5)

({'NDCG:top=1;type=Base': 0.9887640449438202},
 {'NDCG:top=5;type=Base': 0.9897698602053552})

In [97]:
metrics_test_at(1), metrics_test_at(5)

({'NDCG:top=1;type=Base': 0.9833333333333333},
 {'NDCG:top=5;type=Base': 0.9948904400602878})

In [98]:
# очищаем ОЗУ
del model
del full_df
del train_df, val_df, test_df
del train, val, test
del X_train, y_train, g_train
del X_val, y_val, g_val
del X_test, y_test, g_test

In [99]:
end_time = time.time()
total_time = (end_time - start_time) / 60
print(f'Общее время работы baseline: {int(total_time)} минут')

Общее время работы baseline: 117 минут
